## Import all necessary packages

In [7]:
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px

## Import rain data

In [5]:
rain = pd.read_csv('../data/rain.csv')
rain = rain.dropna()

(305, 2)
         Date  Rainfall
0      1/1/20       0.0
1      2/1/20       0.2
2      3/1/20       0.0
3      4/1/20       0.0
4      5/1/20       0.0
..        ...       ...
300  27/10/20      13.6
301  28/10/20       0.0
302  29/10/20      15.3
303  30/10/20       0.0
304  31/10/20       2.8

[305 rows x 2 columns]


## Convert the date to a proper datetime format

In [6]:
import datetime
rain['date_modified'] = [datetime.date(int(str(val).split('/')[2] +'00'), int(str(val).split('/')[1]),int(str(val).split('/')[0]))  for val in rain['Date']]
rain['date_modified'] = pd.to_datetime(rain['date_modified'])
print(rain)

         Date  Rainfall date_modified
0      1/1/20       0.0    2000-01-01
1      2/1/20       0.2    2000-01-02
2      3/1/20       0.0    2000-01-03
3      4/1/20       0.0    2000-01-04
4      5/1/20       0.0    2000-01-05
..        ...       ...           ...
300  27/10/20      13.6    2000-10-27
301  28/10/20       0.0    2000-10-28
302  29/10/20      15.3    2000-10-29
303  30/10/20       0.0    2000-10-30
304  31/10/20       2.8    2000-10-31

[305 rows x 3 columns]


In [51]:
# Create the buttons
date_buttons = [
{'count': 28, 'label': '4WTD', 'step': "day", 'stepmode': "todate"},
{'count': 48, 'label': '48HR', 'step': "hour", 'stepmode': "todate"},
{'count': 1, 'label': 'YTD', 'step': "year", 'stepmode': "todate"}]

# Create the basic line chart
fig = px.line(data_frame=rain, x='date_modified', y='Rainfall', 
              title="Rainfall (mm)")

# Add the buttons and show
fig.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=list([
            dict(count=28, label="4WTD", step="day", stepmode="todate"),
            dict(count=48, label="48HR", step="hour", stepmode="todate"),
            dict(count=1, label="YTD", step="year", stepmode="todate"),
            dict(step="all")
        ])
    )
)

fig.show()

In [38]:
# Add a new column Month to store the month
rain["Month"] = rain["date_modified"].dt.month
# Create a simple bar chart
fig = px.bar(data_frame=rain, x='Month', y='Rainfall')

# Create the buttons
my_buttons = [{'label': "Bar plot", 'method': "update", 'args': [{"type": "bar"}]},
  {'label': "scatterplot", 'method': "update", 'args': [{"type": "scatter", 'mode': 'markers'}]}]

# Add buttons to the plot and show
fig.update_layout({
    'updatemenus': [{
      'type':'buttons','direction': 'down',
      'x': 1.3,'y': 0.5,
      'showactive': True, 'active': 0,
      'buttons': my_buttons}]})
fig.show()

## Import Sales Data

In [50]:
sales = pd.read_csv('../data/monthly_sales.csv')
sales = sales.dropna()
#print(sales)

# Create the basic figure
fig = go.Figure()

# Add a trace per metric
fig.add_trace(go.Bar(x=sales["Month"], y=sales['Sales Volume'], name='Sales Volume'))
fig.add_trace(go.Bar(x=sales["Month"], y=sales['Sales Value'], name='Sales Value'))

# Create annotations
value_annotations=[{'text': 'Sept was the best' ,'showarrow': True, 'x': 'September', 'y': 345397 }]
volume_annotations=[{'text': 'Oct was the best', 'showarrow': True, 'x': 'October', 'y': 71900 }]

#dir(fig.layout)

# Create buttons
my_buttons = [
{'label': "By Sales Value", 'method': "update", 'args': [{}, {'annotations': value_annotations}]},
{'label': "By Sales Volume", 'method': "update", 'args': [{}, {'annotations': volume_annotations}]}]

# Add the buttons
fig.update_layout({
    'updatemenus':[{
            'type': "buttons",
            'direction': 'down',
            'x': 1.3,'y': 0.5, 'buttons': my_buttons
            }]})
fig.show()

## Create a new data frame with total rains per month

In [75]:
def getMonth(nummonth):
    
    match nummonth:
        case 12 | 1 | 2:
            return "Summer"
        case 3 | 4 | 5:
            return "Autumn"
        case 6 | 7 | 8:
            return "Winter"
        case 9 | 10 | 11:
            return "Spring"
    
# add a new columns to store the month
rain['Month'] = [getMonth(val.month) for val in rain['date_modified']]

rain_pm = rain.groupby('Month').agg({'Rainfall':['sum']}).reset_index()
rain_pm.columns = rain_pm.columns.map('_'.join)
rain_pm = rain_pm.rename(columns={'Month_': 'Season', 'Rainfall_sum': 'Total Rainfall'})

# Create the basic figure
fig = go.Figure()

# Loop through the Sessions
for season in ['Autumn', 'Winter', 'Spring','Summer']:
  	# Subset the DataFrame
    df = rain_pm[rain_pm.Season == season]
    # Add a trace for each season
    fig.add_trace(go.Bar(x=df['Season'], y=df['Total Rainfall'], name=season))
    
    # Create the slider elements
sliders = [
    {'steps':[
    {'method': 'update', 'label': 'Autumn', 'args': [{'visible': [True, False, False, False]}]},
    {'method': 'update', 'label': 'Winter', 'args': [{'visible': [False, True, False, False]}]},
    {'method': 'update', 'label': 'Spring', 'args': [{'visible': [False, False, True, False]}]},
    {'method': 'update', 'label': 'Summer', 'args': [{'visible': [False, False, False, True]}]}    
    ]}]

# Update the figure to add sliders and show
fig.update_layout({'sliders': sliders})

# Show the plot
fig.show()